# UC-SLS Lecture 20 : Using LibC to access the OS and escape the confines our our process

- Address Space management:
  - dynamic memory for data items: `malloc` and `free`
  - more powerfully control using `mmap` and `munmap`
- I/O
  - low-level file descriptor based: `open`, `read`, `write`, `close`, `getc`, `getchar`, `gets`, `putc`, `putchar`, `puts`
  - Formatted and Buffered IO: `fopen`, `fread`, `fwrite`, `fclose`, `fprintf`/`printf`,
  `fscanf`/`scanf`, `fgetc`, `fgets`, `fputs`, `fputc`, `fgetpos`, `fsetpos`, `fseek`, `fflush`, 